In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

In [2]:
# #weather data preprocessing (for colusa county, few fires)

# #cols to keep, that have >2000 samples
# selected_col_labels = ['Station', 'Date', 'Time', 'Interval', 'Air max', 'Air max time',
#        'Air min', 'Air obs', 'Air3 avg', 'RH max time', 'RH min time',
#        'Soil avg', 'Precip', 'ETo', 'Leaf3', 'Wind max dir', 'Wind max speed',
#        'Battery', 'Hrs 26F', 'Hrs 28F', 'HR65TO75FA90RH3']
# #all col labels
# col_labels = ["Station","Date","Time","Interval","Air max","Air max time","Air min",
#               "Air min time","Air obs","Air avg","Air2 max","Air2 max time","Air2 min",
#               "Air2 min time","Air2 obs","Air2 avg","Air3 max","Air3 max time","Air3 min",
#               "Air3 min time","Air3 obs","Air3 avg","RH max","RH max time","RH min",
#               "RH min time","RH avg","RH2 max","RH2 max time","RH2 min","RH2 min time",
#               "RH2 avg","RH3 max","RH3 max time","RH3 min","RH3 min time","RH3 avg",
#               "Soil max","Soil max time","Soil min","Soil min time","Soil obs","Soil avg",
#               "Precip","ETo","Solar","Leaf","Leaf2","Leaf3","Wind max dir","Wind max speed",
#               "Wind max time","Wind obs dir","Wind obs speed","Wind avg dir","Wind avg speed",
#               "Wind ST","Wind run","Battery","Dew pnt","Chill hrs","Hrs 26F","Hrs 28F","Hrs 32F",
#               "DD41F","DD41F2","DD41F3","TC DSV","TC DSV2","TC DSV3","HR65TO75FA90RH","HR65TO75FA90RH2",
#               "HR65TO75FA90RH3","BEEHOURS","BEESEASON"]

# weather_df = pd.read_csv("Colusa Weather Data.csv", na_values=[' ', ''])
# weather_df = weather_df.dropna(axis=1, thresh=2000)
# correct_selected_col_labels_dict = dict(zip(selected_col_labels, [col_labels[col_labels.index(s) + 1] for s in selected_col_labels]))
# weather_df = weather_df.rename(columns=correct_selected_col_labels_dict)
# remove_cols = ['BEEHOURS']
# weather_df = weather_df.drop(columns=remove_cols)
# #print(weather_df.columns)
# display(weather_df)




In [3]:
#weather data preprocessing

# Description below generated by chatGPT
# FEW (Few): 1/8 to 2/8 of the sky covered by clouds.
# SCT (Scattered): 3/8 to 4/8 of the sky covered by clouds.
# BKN (Broken): 5/8 to 7/8 of the sky covered by clouds.
# OVC (Overcast): 8/8 of the sky covered by clouds.
cloud_cover_codes = ['CLR', 'FEW', 'SCT', 'BKN', 'OVC']

weather_df = pd.read_csv("Riverside Weather Data.csv", na_values=[' ', ''])
weather_df = weather_df.dropna(axis=1, thresh=20000)
remove_cols = ['REPORT_TYPE', 'SOURCE', 'HourlyAltimeterSetting', 'REM', 'REPORT_TYPE.1', 'SOURCE.1', 'WindEquipmentChangeDate']
weather_df = weather_df.drop(columns=remove_cols)
for column_name in list(weather_df.columns): 
    if isinstance(weather_df[column_name][0], str): 
        weather_df[column_name] = weather_df[column_name].str.rstrip('s')

#cloud cover processing
cloud_df = weather_df['HourlySkyConditions']
cloud_cover_list = []
for condition in cloud_df.to_list():
    #print(condition)
    if isinstance(condition, str) and condition[0:3] in cloud_cover_codes: #check that entry is not NaN
        cloud_cover_list.append(float(condition.split(':')[1][0:2])/8.0)
    else:
        cloud_cover_list.append(np.nan) #mark NaN values with -1

cloud_cover_df = pd.DataFrame(cloud_cover_list, columns=['cloud cover percentage'])
#cloud_height_range_df =
weather_df = pd.concat([weather_df, cloud_cover_df], axis=1).drop(columns=['HourlySkyConditions'])
weather_df = weather_df.replace('*', np.nan)

#from chatgpt
# Define a function to convert string to float (handles NaN values)
def convert_to_float(value):
    try:
        return float(value)
    except ValueError:
        return np.nan

# Apply the conversion function to all elements in the DataFrame
print(float(weather_df['HourlyDewPointTemperature'][0]))
display(weather_df)
convert_cols = ['HourlyDewPointTemperature', 'HourlyDryBulbTemperature', 'HourlyPrecipitation', 
                'HourlyPressureChange', 'HourlyPressureTendency', 'HourlyRelativeHumidity', 
                'HourlySeaLevelPressure', 'HourlyStationPressure', 'HourlyVisibility', 
                'HourlyWetBulbTemperature', 'HourlyWindDirection', 'HourlyWindSpeed', 
                'cloud cover percentage']
weather_df = weather_df.replace('T', np.nan)
weather_df = weather_df.replace('s', '', regex=True)
weather_df = weather_df.replace('V', '', regex=True)
#from chatgpt

#\from chatgpt
for convert_col in convert_cols :
    for value, index in zip(weather_df[convert_col], range(len(weather_df[convert_col]))):
        weather_df.loc[index, convert_col] = convert_to_float(value)
        
#\from chatgpt
display(weather_df)
#we need to convert from strings to floats before interpolating
weather_df = weather_df.interpolate()
weather_df = weather_df[weather_df['DATE'].str.contains('T12')]  #only take one sample at ~midday every day by dropping other rows
display(weather_df)

C:\Users\Isaac\AppData\Local\Temp\ipykernel_19728\689494883.py:10: DtypeWarning: Columns (21,27,28,29,30,31,37,38,49,50,54,56,57,59,60,62,64,116,118,123) have mixed types. Specify dtype option on import or set low_memory=False.
  weather_df = pd.read_csv("Riverside Weather Data.csv", na_values=[' ', ''])


12.0


,STATION,DATE,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyPressureChange,HourlyPressureTendency,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyStationPressure,HourlyVisibility,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindSpeed,cloud cover percentage
0,72286903171,2016-01-01T00:53:00,12,48,0.00,0.02,8.0,23.0,30.07,29.21,10.00,35.0,060,7.0,0.0
1,72286903171,2016-01-01T01:53:00,12,49,0.00,NaN,NaN,22.0,30.07,29.21,10.00,36.0,050,8.0,0.0
2,72286903171,2016-01-01T02:53:00,13,46,0.00,NaN,NaN,26.0,30.07,29.21,10.00,34.0,080,6.0,0.0
3,72286903171,2016-01-01T03:53:00,12,47,0.00,0.01,8.0,24.0,30.06,29.19,10.00,35.0,070,8.0,0.0
4,72286903171,2016-01-01T04:53:00,13,45,0.00,NaN,NaN,27.0,30.07,29.20,10.00,34.0,080,7.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80674,72286903171,2023-12-01T19:53:00,41,54,NaN,NaN,NaN,62.0,30.04,29.18,10.00,48.0,080,3.0,0.0
80675,72286903171,2023-12-01T20:53:00,41,52,NaN,NaN,NaN,66.0,30.05,29.19,10.00,47.0,070,3.0,0.0
80676,72286903171,2023-12-01T21:53:00,40,50,NaN,-0.05,3.0,68.0,30.08,29.21,10.00,45.0,000,0.0,0.0
80677,72286903171,2023-12-01T22:53:00,40,49,NaN,NaN,NaN,71.0,30.07,29.21,10.00,45.0,050,5.0,0.0


,STATION,DATE,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyPressureChange,HourlyPressureTendency,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyStationPressure,HourlyVisibility,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindSpeed,cloud cover percentage
0,72286903171,2016-01-01T00:53:00,12.0,48.0,0.0,0.02,8.0,23.0,30.07,29.21,10.0,35.0,60.0,7.0,0.0
1,72286903171,2016-01-01T01:53:00,12.0,49.0,0.0,NaN,NaN,22.0,30.07,29.21,10.0,36.0,50.0,8.0,0.0
2,72286903171,2016-01-01T02:53:00,13.0,46.0,0.0,NaN,NaN,26.0,30.07,29.21,10.0,34.0,80.0,6.0,0.0
3,72286903171,2016-01-01T03:53:00,12.0,47.0,0.0,0.01,8.0,24.0,30.06,29.19,10.0,35.0,70.0,8.0,0.0
4,72286903171,2016-01-01T04:53:00,13.0,45.0,0.0,NaN,NaN,27.0,30.07,29.20,10.0,34.0,80.0,7.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80674,72286903171,2023-12-01T19:53:00,41.0,54.0,NaN,NaN,NaN,62.0,30.04,29.18,10.0,48.0,80.0,3.0,0.0
80675,72286903171,2023-12-01T20:53:00,41.0,52.0,NaN,NaN,NaN,66.0,30.05,29.19,10.0,47.0,70.0,3.0,0.0
80676,72286903171,2023-12-01T21:53:00,40.0,50.0,NaN,-0.05,3.0,68.0,30.08,29.21,10.0,45.0,0.0,0.0,0.0
80677,72286903171,2023-12-01T22:53:00,40.0,49.0,NaN,NaN,NaN,71.0,30.07,29.21,10.0,45.0,50.0,5.0,0.0


C:\Users\Isaac\AppData\Local\Temp\ipykernel_19728\689494883.py:62: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  weather_df = weather_df.interpolate()


,STATION,DATE,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyPressureChange,HourlyPressureTendency,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyStationPressure,HourlyVisibility,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindSpeed,cloud cover percentage
12,72286903171,2016-01-01T12:53:00,12.0,62.0,0.0,0.060000,8.000000,14.0,30.07,29.21,10.0,43.0,NaN,5.0,0.500
37,72286903171,2016-01-02T12:53:00,18.0,68.0,0.0,0.070000,8.000000,15.0,30.0,29.14,10.0,47.0,0.0,0.0,0.000
62,72286903171,2016-01-03T12:53:00,34.0,62.0,0.0,0.080000,8.000000,35.0,29.89,29.03,10.0,49.0,280.0,10.0,0.000
87,72286903171,2016-01-04T12:53:00,37.0,65.0,0.0,0.040000,8.000000,36.0,29.74,28.89,10.0,51.0,NaN,3.0,1.000
122,72286903171,2016-01-05T12:21:00,52.0,55.0,0.03,0.045714,5.428571,90.0,NaN,28.92,2.0,53.0,110.0,5.0,0.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80546,72286903171,2023-11-27T12:53:00,21.0,69.0,0.0,0.050000,8.000000,16.0,30.02,29.17,10.0,48.0,20.0,10.0,0.000
80571,72286903171,2023-11-28T12:53:00,29.0,72.0,0.0,0.090000,8.000000,20.0,30.04,29.19,10.0,52.0,260.0,11.0,0.000
80596,72286903171,2023-11-29T12:53:00,41.0,64.0,0.0,0.090000,8.000000,43.0,29.92,29.06,10.0,52.0,250.0,8.0,0.250
80631,72286903171,2023-11-30T12:53:00,47.0,61.0,0.0,0.070000,8.000000,60.0,29.86,29.01,10.0,54.0,0.0,0.0,0.875


In [6]:
#fire data preprocessing
fires_df = pd.read_csv("mapdataall.csv")
#print(len(fires_df))
riverside_fires = fires_df[fires_df['incident_county'] == 'Riverside']
riverside_fire_dates = riverside_fires['incident_date_created'].apply(lambda x: x.split('T')[0])
#print(riverside_fire_dates)
#print(len(riverside_fires))

In [8]:
riverside_fire_dates = list(riverside_fire_dates)
#print(riverside_fire_dates)
Y = []
for date in weather_df['DATE']:
    date = date.split('T')[0]
    value = True if date in riverside_fire_dates else False
    Y.append(value)
Y = np.array(Y)

#print(Y)

#more data processing
drop_label_cols = ['STATION', 'DATE']
weather_df_with_dates = copy.copy(weather_df)
weather_df = weather_df.drop(columns=drop_label_cols)

# for column_name in list(weather_df.columns): 
#     for i in range(len(weather_df[column_name])):
#         if isinstance(weather_df[column_name][i], str) and weather_df[column_name][i] != np.nan: 
#             weather_df[column_name][i] = float(weather_df[column_name][i])



X = weather_df

print(weather_df.isnull().sum())
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

#use a scitkit learn scaler so that we can use it later for new data points:
scaler_input = StandardScaler()
scaler_input.fit(X_train)
data_processed = scaler_input.transform(X_train)

#put the scaled data back into a dataframe
data_processed = pd.DataFrame(data_processed)
data_processed.index = X_train.index
mapcols = {}
for idx in range(len(X_train.columns)):
    mapcols[idx] = X_train.columns[idx]
X_train = data_processed.rename(columns=mapcols)

model = LogisticRegression()

#from chatgpt

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test data (probabilities)
y_proba = model.predict_proba(X_test)

#\from chatgpt

# Extract the probability of belonging to class 1
prob_class_1 = dict(zip([s.split('T')[0] for s in weather_df_with_dates['DATE'].tolist()], y_proba[:, 1]))
print(np.mean(np.array(list(prob_class_1.values()))))
print(np.mean(y_proba[:,1]))
#print([prob_class_1[s] for s in riverside_fire_dates])

KeyError: 'DATE'

In [ ]:
# df = pd.read_csv("mapdataall.csv")
# months = df["incident_date_created"].str.slice(5,7)
# days = df["incident_date_created"].str.slice(8,10)
# counts = months.value_counts()
# #print(counts)
# ##code from chatgpt
# # Use zip to pair elements from both lists
# zipped_lists = list(zip(counts.index, counts))

# # Sort the zipped list based on the values from list1
# sorted_zipped_lists = sorted(zipped_lists, key=lambda x: x[0])

# # Extract the sorted elements back into separate lists
# indices, sorted_counts = zip(*sorted_zipped_lists)
# ##\code from chatgpt
# plt.plot(indices, sorted_counts)